In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import requests

In [2]:
# Charger le modèle et le processeur
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

c:\Users\benja\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\benja\.cache\huggingface\hub\models--timm--resnet50.a1_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

c:\Users\benja\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\benja\.cache\huggingface\hub\models--facebook--detr-resnet-50. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [23]:
# Charger une image
image = Image.open("Lionel-Messi-Kylian-Mbappe-Paris-gagnant.jpg")

In [24]:
# Préparer l'image et faire des prédictions
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [25]:
# Extraire les résultats
logits = outputs.logits
probas = logits.softmax(-1)
boxes = outputs.pred_boxes

In [26]:
from PIL import ImageDraw

# Créer un objet de dessin pour l'image
draw = ImageDraw.Draw(image)

# Classe ID de "person"
id2label = model.config.id2label  # Map des IDs aux labels
person_id = [k for k, v in id2label.items() if v == "person"][0]

# Réduire la première dimension (batch size)
probas = probas[0]
boxes = boxes[0]

for i, (box, scores) in enumerate(zip(boxes, probas)):
    # Extraire la probabilité pour la classe "person"
    score = scores[person_id].item()

    if score > 0.5:  # Seuil de confiance
        # Convertir les coordonnées en pixels
        box = box.detach().numpy() * [image.width, image.height, image.width, image.height]
        x_min, y_min, x_max, y_max = map(int, box)

        # Vérifier et corriger les coordonnées
        x_min = max(0, min(image.width, x_min))
        y_min = max(0, min(image.height, y_min))
        x_max = max(0, min(image.width, x_max))
        y_max = max(0, min(image.height, y_max))

        # S'assurer que les coordonnées forment une boîte valide
        if x_min < x_max and y_min < y_max:
            # Dessiner le rectangle
            draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)
            draw.text((x_min, y_min), f"Person: {score:.2f}", fill="red")


# Afficher ou sauvegarder l'image annotée
image.show()  # Pour ouvrir dans une visionneuse d'images
# image.save("output.jpg")  # Pour enregistrer l'image annotée


In [9]:
# Afficher les prédictions
for proba, box in zip(probas[0, :, :-1], boxes[0]):
    if proba.max() > 0.9:
        processor.draw_boxes(image, box)

AttributeError: 'DetrImageProcessor' object has no attribute 'draw_boxes'